In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Reload all modules imported with %aimport
%load_ext autoreload
%autoreload 1

In [2]:
import pandas as pd
idx = pd.IndexSlice

from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.linear_model import LinearRegression

from datetime import timedelta

%aimport trans.data
%aimport trans.gtrans
%aimport trans.reg

from trans.data import GetData
gd = GetData()
from trans.gtrans import *
from trans.reg import Reg


pctTrans     = DataFrameFunctionTransformer(func = lambda s: s.pct_change())
rankTrans    = DataFrameFunctionTransformer(func = lambda s: s.rank(method="first"), axis=1)
pctOnlyTrans = GenSelectAttrsTransformer(['Pct'], dropSingle=False )

## Get the raw data

In [3]:
%aimport trans.data
raw_df = gd.combine_data(['FB', 'AAPL', 'AMZN', 
                           'NFLX', 'GOOG', 'SPY'])
raw_df.head()

0


Open                                          High            \
             FB      AAPL     AMZN NFLX GOOG         SPY   FB      AAPL   
Date                                                                      
2000-01-03  NaN  3.745536  81.5000  NaN  NaN  148.250000  NaN  4.017857   
2000-01-04  NaN  3.866071  85.3750  NaN  NaN  143.531204  NaN  3.950893   
2000-01-05  NaN  3.705357  70.5000  NaN  NaN  139.937500  NaN  3.948661   
2000-01-06  NaN  3.790179  71.3125  NaN  NaN  139.625000  NaN  3.821429   
2000-01-07  NaN  3.446429  67.0000  NaN  NaN  140.312500  NaN  3.607143   

                             ...     Adj Close                       Volume  \
               AMZN NFLX     ...          AMZN NFLX GOOG         SPY     FB   
Date                         ...                                              
2000-01-03  89.5625  NaN     ...       89.3750  NaN  NaN  103.807343    NaN   
2000-01-04  91.5000  NaN     ...       81.9375  NaN  NaN   99.747833    NaN   
2000-01-05  75.1250  NaN     ...       69.7500  NaN  NaN   99.926254    NaN   
2000-01-06  72.6875  NaN     ...       65.5625  NaN  NaN   98.320305    NaN   
2000-01-07  70.5000  NaN     ...       69.5625  NaN  NaN  104.030365    NaN   

                                                           
                   AAPL        AMZN NFLX GOOG         SPY  
Date                                                       
2000-01-03  133949200.0  16117600.0  NaN  NaN   8164300.0  
2000-01-04  128094400.0  17487400.0  NaN  NaN   8089800.0  
2000-01-05  194580400.0  38457400.0  NaN  NaN  12177900.0  
2000-01-06  191993200.0  18752000.0  NaN  NaN   6227200.0  
2000-01-07  115183600.0  10505400.0  NaN  NaN   8066500.0  

[5 rows x 36 columns]

## Define featUn transformer: compute Pct and append to Adj Close

In [4]:
pipe_close = make_pipeline(GenSelectAttrsTransformer(['Adj Close'], dropSingle=True )
                      )   

pipe_pct   = make_pipeline(GenSelectAttrsTransformer(['Adj Close'], dropSingle=True ), 
                         pctTrans,
                      )

featUn = GenDataFrameFeatureUnion( [ ("Adj Close", pipe_close),
                                    ("Pct", pipe_pct)
                                   ] )

In [5]:
pipe_pct_only   = make_pipeline(GenSelectAttrsTransformer(['Pct'], dropSingle=False ) ) 

## Create pipeline to prepare data for regression

In [6]:
pipe_nn = make_pipeline( featUn,
                         DatetimeIndexTransformer("Dt"),
                         pctOnlyTrans,
                         RestrictToCalendarColTransformer( ("Pct", "SPY")),
                         RestrictToNonNullTransformer("all"),
                         # FillNullTransformer(method="bfill")
                       )
pct_df = pipe_nn.fit_transform(raw_df)
pct_df.head()

GenSelect: zip: [('Adj Close', 'FB'), ('Adj Close', 'AAPL'), ('Adj Close', 'AMZN'), ('Adj Close', 'NFLX'), ('Adj Close', 'GOOG'), ('Adj Close', 'SPY')],
	mi: MultiIndex(levels=[['Adj Close'], ['AAPL', 'AMZN', 'FB', 'GOOG', 'NFLX', 'SPY']],
           labels=[[0, 0, 0, 0, 0, 0], [2, 0, 1, 4, 3, 5]])
GenSelect: zip: [('Adj Close', 'FB'), ('Adj Close', 'AAPL'), ('Adj Close', 'AMZN'), ('Adj Close', 'NFLX'), ('Adj Close', 'GOOG'), ('Adj Close', 'SPY')],
	mi: MultiIndex(levels=[['Adj Close'], ['AAPL', 'AMZN', 'FB', 'GOOG', 'NFLX', 'SPY']],
           labels=[[0, 0, 0, 0, 0, 0], [2, 0, 1, 4, 3, 5]])
GenSelect: zip: [('Adj Close', 'FB'), ('Adj Close', 'AAPL'), ('Adj Close', 'AMZN'), ('Adj Close', 'NFLX'), ('Adj Close', 'GOOG'), ('Adj Close', 'SPY')],
	mi: MultiIndex(levels=[['Adj Close'], ['AAPL', 'AMZN', 'FB', 'GOOG', 'NFLX', 'SPY']],
           labels=[[0, 0, 0, 0, 0, 0], [2, 0, 1, 4, 3, 5]])
GenSelect: zip: [('Pct', 'AAPL'), ('Pct', 'AMZN'), ('Pct', 'FB'), ('Pct', 'GOOG'), ('Pct', 'NFLX'), 

Pct                                  
                AAPL      AMZN  FB GOOG NFLX       SPY
Dt                                                    
2000-01-04 -0.084310 -0.083217 NaN  NaN  NaN -0.039106
2000-01-05  0.014634 -0.148741 NaN  NaN  NaN  0.001789
2000-01-06 -0.086538 -0.060036 NaN  NaN  NaN -0.016071
2000-01-07  0.047368  0.061010 NaN  NaN  NaN  0.058076
2000-01-10 -0.017588 -0.005391 NaN  NaN  NaN  0.003431

## Do a rolling regression on the dataframe with prepared data

In [7]:
ra = Reg(pct_df)
ma = ra.modelCols( [ idx["Pct", "SPY"]])
ma

beta_df = ra.rollingModelAll( *ma, #idx["Pct", "AAPL"],
                 pd.to_datetime("01/01/2000",infer_datetime_format=True),
                 # pd.to_datetime("04/14/2000", infer_datetime_format=True),
                 pd.to_datetime("12/29/2017", infer_datetime_format=True),
                 timedelta(weeks=4)
            )
beta_df.tail()

([('Pct', 'SPY')],
 [('Pct', 'AAPL'),
  ('Pct', 'AMZN'),
  ('Pct', 'FB'),
  ('Pct', 'GOOG'),
  ('Pct', 'NFLX')])

Fit: the following columns have naN:  ['FB']
Fit: the following columns have naN:  ['FB']
Fit: the following columns have naN:  ['FB']
Fit: the following columns have naN:  ['FB']
Fit: the following columns have naN:  ['FB']
Fit: the following columns have naN:  ['FB']
Fit: the following columns have naN:  ['FB']
Fit: the following columns have naN:  ['FB']
Fit: the following columns have naN:  ['FB']
Fit: the following columns have naN:  ['FB']
Fit: the following columns have naN:  ['FB']
Fit: the following columns have naN:  ['FB']
Fit: the following columns have naN:  ['FB']
Fit: the following columns have naN:  ['FB']
Fit: the following columns have naN:  ['FB']
Fit: the following columns have naN:  ['FB']
Fit: the following columns have naN:  ['FB']
Fit: the following columns have naN:  ['FB']
Fit: the following columns have naN:  ['FB']
Fit: the following columns have naN:  ['FB']
Fit: the following columns have naN:  ['FB']
Fit: the following columns have naN:  ['FB']
Fit: the f

Beta 0                                            Beta 1  \
                AAPL      AMZN        FB      GOOG      NFLX      AAPL   
Dt                                                                       
2017-09-08 -0.000272 -0.000858  0.000231  0.000056  0.000910  1.285798   
2017-10-06 -0.005136 -0.001802 -0.004168  0.001119  0.000013  2.281003   
2017-11-03  0.003963  0.001401 -0.000019  0.000439 -0.000820  1.733427   
2017-12-01 -0.001837  0.001098 -0.002488 -0.002125 -0.006099  1.339091   
2017-12-29 -0.001793 -0.000217 -0.000183  0.001072  0.000821  1.699265   

                                                    
                AMZN        FB      GOOG      NFLX  
Dt                                                  
2017-09-08  1.464521  1.288074  1.239199  1.292776  
2017-10-06  1.691252  2.566470  0.923909  3.312801  
2017-11-03  6.054137  2.509251  2.880951  1.734489  
2017-12-01  1.082630  1.164516  0.837844  2.177337  
2017-12-29  0.767625  0.859952  1.081924  0.908602

## Append the rolling betas to the prepared data

In [8]:
concatTrans = DataFrameConcat( [ pct_df, beta_df ])
ret_and_beta_df = concatTrans.fit_transform(pd.DataFrame())
ret_and_beta_df.tail()
ret_and_beta_df.shape

Beta 0                    Beta 1                          Pct  \
             AAPL AMZN  FB GOOG NFLX   AAPL AMZN  FB GOOG NFLX      AAPL   
Dt                                                                         
2018-02-01    NaN  NaN NaN  NaN  NaN    NaN  NaN NaN  NaN  NaN  0.002090   
2018-02-02    NaN  NaN NaN  NaN  NaN    NaN  NaN NaN  NaN  NaN -0.043390   
2018-02-05    NaN  NaN NaN  NaN  NaN    NaN  NaN NaN  NaN  NaN -0.024984   
2018-02-06    NaN  NaN NaN  NaN  NaN    NaN  NaN NaN  NaN  NaN  0.041792   
2018-02-07    NaN  NaN NaN  NaN  NaN    NaN  NaN NaN  NaN  NaN -0.011654   

                                                              
                AMZN        FB      GOOG      NFLX       SPY  
Dt                                                            
2018-02-01 -0.041967  0.033175 -0.001915 -0.019349 -0.001135  
2018-02-02  0.028741 -0.014553 -0.047786  0.008903 -0.021770  
2018-02-05 -0.027938 -0.047404 -0.050454 -0.049247 -0.041823  
2018-02-06  0.038014  0.022344  0.023489  0.045072  0.019702  
2018-02-07 -0.007853 -0.017970 -0.017065  0.005419  0.002229

(4561, 16)

In [9]:
gd.save_data(pct_df, "ret_df.pkl")
gd.save_data(beta_df, "beta_df.pkl")
gd.save_data(ret_and_beta_df, "ret_and_beta_df.pkl")

## Prepare for Return Attribution

### Find the attributes with the sensitivities

In [10]:
rab = Reg(ret_and_beta_df)
betaAttrs = rab.sensAttrs('^Beta \d+$')
betaAttrs

['Beta 0', 'Beta 1']

## Roll the betas forward

In [11]:
beta_r_pl = make_pipeline( GenSelectAttrsTransformer(betaAttrs),
                            ShiftTransformer(1),
                            FillNullTransformer(method="ffill"),
                            GenRenameAttrsTransformer(lambda col: col + ' rolled fwd', level=0)
                         )
beta_rolled_df = beta_r_pl.fit_transform(ret_and_beta_df)
beta_rolled_df.tail()

GenSelect: zip: [('Beta 0', 'AAPL'), ('Beta 0', 'AMZN'), ('Beta 0', 'FB'), ('Beta 0', 'GOOG'), ('Beta 0', 'NFLX'), ('Beta 1', 'AAPL'), ('Beta 1', 'AMZN'), ('Beta 1', 'FB'), ('Beta 1', 'GOOG'), ('Beta 1', 'NFLX')],
	mi: MultiIndex(levels=[['Beta 0', 'Beta 1'], ['AAPL', 'AMZN', 'FB', 'GOOG', 'NFLX']],
           labels=[[0, 0, 0, 0, 0, 1, 1, 1, 1, 1], [0, 1, 2, 3, 4, 0, 1, 2, 3, 4]])
transform: pandas version <= 0.20.
Columns level 0: Index(['Beta 0', 'Beta 1'], dtype='object')
New Columns level 0: ['Beta 0 rolled fwd' 'Beta 1 rolled fwd']


Beta 0 rolled fwd                                          \
                        AAPL      AMZN        FB      GOOG      NFLX   
Dt                                                                     
2018-02-01         -0.001793 -0.000217 -0.000183  0.001072  0.000821   
2018-02-02         -0.001793 -0.000217 -0.000183  0.001072  0.000821   
2018-02-05         -0.001793 -0.000217 -0.000183  0.001072  0.000821   
2018-02-06         -0.001793 -0.000217 -0.000183  0.001072  0.000821   
2018-02-07         -0.001793 -0.000217 -0.000183  0.001072  0.000821   

           Beta 1 rolled fwd                                          
                        AAPL      AMZN        FB      GOOG      NFLX  
Dt                                                                    
2018-02-01          1.699265  0.767625  0.859952  1.081924  0.908602  
2018-02-02          1.699265  0.767625  0.859952  1.081924  0.908602  
2018-02-05          1.699265  0.767625  0.859952  1.081924  0.908602  
2018-02-06          1.699265  0.767625  0.859952  1.081924  0.908602  
2018-02-07          1.699265  0.767625  0.859952  1.081924  0.908602

### Append the rolled betas to the regression results

In [12]:
ret_and_rolled_beta_pl = DataFrameConcat( [ ret_and_beta_df, beta_rolled_df])
ret_and_rolled_beta_df = ret_and_rolled_beta_pl.fit_transform( pd.DataFrame() )
ret_and_rolled_beta_df.tail()

Beta 0                    Beta 0 rolled fwd                      \
             AAPL AMZN  FB GOOG NFLX              AAPL      AMZN        FB   
Dt                                                                           
2018-02-01    NaN  NaN NaN  NaN  NaN         -0.001793 -0.000217 -0.000183   
2018-02-02    NaN  NaN NaN  NaN  NaN         -0.001793 -0.000217 -0.000183   
2018-02-05    NaN  NaN NaN  NaN  NaN         -0.001793 -0.000217 -0.000183   
2018-02-06    NaN  NaN NaN  NaN  NaN         -0.001793 -0.000217 -0.000183   
2018-02-07    NaN  NaN NaN  NaN  NaN         -0.001793 -0.000217 -0.000183   

                                  ...    Beta 1 rolled fwd            \
                GOOG      NFLX    ...                 AMZN        FB   
Dt                                ...                                  
2018-02-01  0.001072  0.000821    ...             0.767625  0.859952   
2018-02-02  0.001072  0.000821    ...             0.767625  0.859952   
2018-02-05  0.001072  0.000821    ...             0.767625  0.859952   
2018-02-06  0.001072  0.000821    ...             0.767625  0.859952   
2018-02-07  0.001072  0.000821    ...             0.767625  0.859952   

                                     Pct                                \
                GOOG      NFLX      AAPL      AMZN        FB      GOOG   
Dt                                                                       
2018-02-01  1.081924  0.908602  0.002090 -0.041967  0.033175 -0.001915   
2018-02-02  1.081924  0.908602 -0.043390  0.028741 -0.014553 -0.047786   
2018-02-05  1.081924  0.908602 -0.024984 -0.027938 -0.047404 -0.050454   
2018-02-06  1.081924  0.908602  0.041792  0.038014  0.022344  0.023489   
2018-02-07  1.081924  0.908602 -0.011654 -0.007853 -0.017970 -0.017065   

                                
                NFLX       SPY  
Dt                              
2018-02-01 -0.019349 -0.001135  
2018-02-02  0.008903 -0.021770  
2018-02-05 -0.049247 -0.041823  
2018-02-06  0.045072  0.019702  
2018-02-07  0.005419  0.002229  

[5 rows x 26 columns]

In [13]:
reg = Reg(ret_and_rolled_beta_df)

### Find the columns for: 
#### independent variables
#### dependent variables
#### sensitivities

In [14]:
indCols = [ ("Pct", "1"), ("Pct", "SPY")]
indCols

sensAttrs = reg.sensAttrs('^Beta \d+ rolled fwd$')
sensAttrs

depTickers = reg.depTickersFromSensAttrs(sensAttrs )
depTickers
depCols = [ ("Pct", t) for t in depTickers ]
depCols



[('Pct', '1'), ('Pct', 'SPY')]

['Beta 0 rolled fwd', 'Beta 1 rolled fwd']

['AAPL', 'AMZN', 'FB', 'GOOG', 'NFLX']

[('Pct', 'AAPL'),
 ('Pct', 'AMZN'),
 ('Pct', 'FB'),
 ('Pct', 'GOOG'),
 ('Pct', 'NFLX')]

## Add constant (for interecept return) column

In [15]:
reg.addConst(("Pct", "1"), 1)

### Perform the return attribution

In [16]:
retAttr_df =reg.retAttrib(
            indCols,
            depCols, 
            sensAttrs)


In [17]:
retAttr_df.tail()

Contrib from 1                                          \
                     AAPL      AMZN        FB      GOOG      NFLX   
Dt                                                                  
2018-02-01      -0.001793 -0.000217 -0.000183  0.001072  0.000821   
2018-02-02      -0.001793 -0.000217 -0.000183  0.001072  0.000821   
2018-02-05      -0.001793 -0.000217 -0.000183  0.001072  0.000821   
2018-02-06      -0.001793 -0.000217 -0.000183  0.001072  0.000821   
2018-02-07      -0.001793 -0.000217 -0.000183  0.001072  0.000821   

           Contrib from SPY                                         Predicted  \
                       AAPL      AMZN        FB      GOOG      NFLX      AAPL   
Dt                                                                              
2018-02-01        -0.001929 -0.000871 -0.000976 -0.001228 -0.001031 -0.003722   
2018-02-02        -0.036993 -0.016711 -0.018721 -0.023553 -0.019780 -0.038785   
2018-02-05        -0.071068 -0.032104 -0.035965 -0.045249 -0.038000 -0.072860   
2018-02-06         0.033479  0.015124  0.016943  0.021316  0.017901  0.031687   
2018-02-07         0.003788  0.001711  0.001917  0.002412  0.002026  0.001996   

                                                       Error            \
                AMZN        FB      GOOG      NFLX      AAPL      AMZN   
Dt                                                                       
2018-02-01 -0.001088 -0.001160 -0.000156 -0.000210  0.005812 -0.040879   
2018-02-02 -0.016928 -0.018904 -0.022481 -0.018959 -0.004605  0.045669   
2018-02-05 -0.032321 -0.036149 -0.044177 -0.037179  0.047876  0.004383   
2018-02-06  0.014907  0.016760  0.022389  0.018723  0.010105  0.023108   
2018-02-07  0.001494  0.001734  0.003484  0.002847 -0.013650 -0.009347   

                                          
                  FB      GOOG      NFLX  
Dt                                        
2018-02-01  0.034334 -0.001759 -0.019139  
2018-02-02  0.004352 -0.025305  0.027862  
2018-02-05 -0.011255 -0.006278 -0.012068  
2018-02-06  0.005584  0.001101  0.026349  
2018-02-07 -0.019704 -0.020549  0.002572

In [18]:
gd.save_data(retAttr_df, "retattr_df.pkl")